In [2]:
import sys
import pandas as pd

In [7]:
users = pd.read_csv('meo_followers.csv')
users.head()
users = users.rename(columns ={'0':'users_id'})
users.head()


,Unnamed: 0,users_id
0,0,448810512
1,1,701174229140639744
2,2,1143603512124878848
3,3,1291226196
4,4,1237638084281409536


In [27]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import tweepy
import datetime
import xlsxwriter
import sys

# credentials from https://apps.twitter.com/
consumerKey = "xMqbF6gGhWHWPUg9xw5ziJZpK"
consumerSecret = "RPGgCCRFD2VpyfcPpEVjPbCYPlFIi2cTEImz6BNWzkIXGekibx"
accessToken = "1119208863574708224-FwHs7oMskXgPAocqA81D5HrOdwV6tA"
accessTokenSecret = "hmlim46RNtaHkFBIHWYYKACAKsUtvsu59YViEjW0bnZ2J"

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)

api = tweepy.API(auth)

username = users['users_id'][15]
startDate = datetime.datetime(2020, 1, 1, 0, 0, 0)
endDate =   datetime.datetime(2020, 2, 1, 0, 0, 0)

tweets = []
tmpTweets = api.user_timeline(username)
for tweet in tmpTweets:
    if tweet.created_at < endDate and tweet.created_at > startDate:
        tweets.append(tweet)

while (tmpTweets[-1].created_at > startDate):
    print("Last Tweet @", tmpTweets[-1].created_at, " - fetching some more")
    tmpTweets = api.user_timeline(username, max_id = tmpTweets[-1].id)
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)

tweets_pd = pd.Series(tweets)

Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet @ 2020-01-17 13:02:54  - fetching some more
Last Tweet

KeyboardInterrupt: 

In [29]:
outtweets = [[tweet.id_str, tweet.created_at, tweet.coordinates,tweet.geo,tweet.source,tweet.text] for tweet in tweets]
type(tweet)

tweepy.models.Status

In [30]:
import json

class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        '---YOUR-KEY---',
        'consumer_secret':     '---YOUR-KEY---',
        'access_token_key':    '---YOUR-KEY---',
        'access_token_secret': '---YOUR-KEY---'
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dril", #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

In [31]:
data

NameError: name 'data' is not defined